In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib as plb
import seaborn as sns
from scipy import stats
import statistics
import dabest as db


#### 1. Unblinding the metadata and cleaning data entry errors
<p>All of our compounds were blinded and placed into 96 well plates for dispensing. We blinded our compounds by assigning them randomized locations within the 96-well plate format and used used the locations as the blinding key.</p>

In [3]:
p1 = pd.read_csv('C:/Users/Emily/Documents/S1F1/S1fuPlate1.csv')
p1s = p1.iloc[range(6)]
p1s = p1s.melt(id_vars=['Let'], var_name= 'Col', value_name= 'Compound')
p1s['CPID'] = 'P1'

In [4]:
p2 = pd.read_csv('C:/Users/Emily/Documents/S1F1/S1fuPlate2.csv')
p2s = p2.iloc[range(6)]
p2s = p2s.melt(id_vars=['Let'], var_name= 'Col', value_name= 'Compound')
p2s['CPID'] = 'P2'

In [5]:
sub = ['Compound', 'Location']
ckey = pd.read_csv('C:/Users/Emily/Documents/S1F1/ckey.csv', index_col=0).drop(columns=['Let','Col'])
ckey_x = pd.read_csv('C:/Users/Emily/Documents/S1F1/D5P1.csv', usecols=sub)
ckey_x.rename(columns={'Location': 'Compound Well'}, inplace=True)
ckey_x['CPID'] = 'D5P1'
ckeys = ckey.append(ckey_x)

<p>There were multiple datasheets that were generated that need to be merged together. We also need to tidy up some of the data entry errors that occured during data collection.

In [6]:
s1f1_md = pd.read_csv('C:/Users/Emily/Documents/S1F1/S1F1_metadata.csv', delimiter=',', 
                      encoding='utf-8-sig')

s1f1_md['Image ID'] = s1f1_md['Image ID'].str[:5] + '0' + s1f1_md['Image ID'].str[5:8]

s1f2_md = pd.read_csv('C:/Users/Emily/Documents/S1F2/S1F2_metadata.csv', delimiter=',', 
                 encoding='utf-8-sig')

all_md = s1f1_md.append(s1f2_md)
all_md.rename(columns={'-': 'Date'}, inplace=True)

The following splits the plate ID string to use to reference the correct plate map for unblinding the data

In [7]:
def add_cipd(row):
    if row['Compound library ID'] != 'D5P1':
        pid = row['Compound library ID'][2:4]
    else:
        pid = 'D5P1'
    return pid

all_md['CPID'] = all_md.apply(lambda row: add_cipd(row), axis=1)


In [8]:
#casting to the correct datatype
all_md['Scanner Slot:'] = all_md['Scanner Slot:'].apply(str)

In [9]:
#Making sure that there isn't any unwanted whitespace in the data entry. Also taking care of wells where no compound was added
df_obj = all_md.select_dtypes(['object', 'string'])
all_md = all_md[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
all_md = all_md.fillna('Empty')
all_md.tail()

,Date,Recorder,Plate ID,Compound library ID,Compound Well A,Compound Well B,Compound Well C,Compound Well D,Chemotaxis Start (24 hrs format),Chemotaxis End (24 hrs format),Image ID,Scanner Slot:,Strain Well A,Strain Well B,Strain Well C,Strain Well D,CPID
34,3/17/2022,Ehsan,S1F2_R3_11,D3P1,F2,F3,F4,F5,2:45,3:45,S1F2_009,3,GN1077,GN1077,GN1077,GN1077,P1
35,3/17/2022,Ehsan,S1F2_R3_12,D3P1,G2,G3,G4,G5,2:45,3:45,S1F2_009,4,GN1077,GN1077,GN1077,GN1077,P1
36,3/21/2022,EMILY,S1F2_R4_1,D4P1,F5,G3,G2,F4,10:55,11:55,S1F2_010,1,GN1077,GN1077,GN1077,GN1077,P1
37,3/21/2022,EMILY,S1F2_R4_2,D4P1,E2,C4,E4,B3,10:55,11:55,S1F2_010,2,GN1077,GN1077,GN1077,GN1077,P1
38,3/21/2022,EMILY,S1F2_R4_3,D4P2,D4,F2,Empty,Empty,10:55,11:55,S1F2_010,3,GN1077,GN1077,GN1077,GN1077,P2


#### Generating functions to unblind the compounds used in each well

In [10]:
def add_compoundA(row, compound_map):
    if row['Compound Well A'] != 'Empty':
        compound = compound_map.loc[
            (compound_map['CPID']==row['CPID']) & 
            (compound_map['Compound Well']==row['Compound Well A'])]['Compound']
        if len(compound)<0:
            return "Empty"
        else:
            return compound.values[0]
    else:
        return "Empty"
    
def add_compoundB(row, compound_map ):
    if row['Compound Well B'] != 'Empty':
        compound = compound_map.loc[
            (compound_map['CPID']==row['CPID']) & 
            (compound_map['Compound Well']==row['Compound Well B'])]['Compound']
        if len(compound)<0:
            return "Empty"
        else:
            return compound.values[0]
    else:
        return "Empty"

def add_compoundC(row, compound_map ):
    if row['Compound Well C'] != 'Empty':
        compound = compound_map.loc[
            (compound_map['CPID']==row['CPID']) & 
            (compound_map['Compound Well']==row['Compound Well C'])]['Compound']
        if len(compound)<0:
            return "Empty"
        else:
            return compound.values[0]
    else:
        return "Empty"

def add_compoundD(row, compound_map ):
    if row['Compound Well D'] != 'Empty':
        compound = compound_map.loc[
            (compound_map['CPID']==row['CPID']) & 
            (compound_map['Compound Well']==row['Compound Well D'])]['Compound']
        if len(compound)<0:
            return "Empty"
        else:
            return compound.values[0]
    else:
        return "Empty"


#### Applying those functions

In [11]:
all_md['Compound A'] = all_md.apply(
    lambda row: add_compoundA(row, ckeys), axis=1)

all_md['Compound B'] = all_md.apply(
    lambda row: add_compoundB(row, ckeys), axis=1)
    
all_md['Compound C'] = all_md.apply(
    lambda row: add_compoundC(row, ckeys), axis=1)
    
all_md['Compound D'] = all_md.apply(
    lambda row: add_compoundD(row, ckeys), axis=1)

#### Reading in and cleaning up the summaries of the image analysis results

In [12]:
F1_ia_results = pd.read_csv('C:/Users/Emily/Documents/S1f1/124_ia/S1F1.csv', index_col=0).drop(columns=['Large Object'])
F2_ia_results = pd.read_csv('C:/Users/Emily/Documents/S1F2/S1F2_ia.csv', index_col=0).drop(columns=['Large Object'])

all_ia = F1_ia_results.append(F2_ia_results)


Some extra images were taken and analyzed when we performed the batch image analysis. These images were not part of the experiment and were not intended to be analyzed in this batch.

In [13]:
broll = ['S1F1_b010', 'S1F1_b011', 'S1F1_b012']
all_ia= all_ia[~all_ia['File Name'].isin(broll)]
ia_results = all_ia.loc[all_ia['Total Worms'] >= 150]
ia_results.head()

,WellNo,Total Worms,Chemotaxis,Compound,Strain,File Name,Well width,Plate ID
1,1B,199.0,0.233533,NaN,NaN,S1F1_001,3044.0,NaN
2,1C,306.0,0.225352,NaN,NaN,S1F1_001,3044.0,NaN
3,1D,208.0,0.390374,NaN,NaN,S1F1_001,3057.0,NaN
4,2A,282.0,-0.019920,NaN,NaN,S1F1_001,3057.0,NaN
5,2B,239.0,0.171429,NaN,NaN,S1F1_001,3043.0,NaN


Creating a function to add the plate id from the metadata to the summary of the results

In [14]:
def add_PlateID(row, metadata):
    slotID = row['WellNo'][0]
    pid = metadata.loc[
        (metadata['Image ID']==row['File Name']) & 
        (metadata['Scanner Slot:']==slotID)]['Plate ID']

    if len(pid) == 0:
        print(row['File Name'])
        pass
    #print(pid)
    else:
        return pid.values[0]

ia_results['Plate ID'] = ia_results.apply(
    lambda row: add_PlateID(row, all_md), axis=1)

ia_results.head()

C:\Users\Emily\miniconda3\envs\neuroplant\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,WellNo,Total Worms,Chemotaxis,Compound,Strain,File Name,Well width,Plate ID
1,1B,199.0,0.233533,NaN,NaN,S1F1_001,3044.0,S1F1_R1_01
2,1C,306.0,0.225352,NaN,NaN,S1F1_001,3044.0,S1F1_R1_01
3,1D,208.0,0.390374,NaN,NaN,S1F1_001,3057.0,S1F1_R1_01
4,2A,282.0,-0.019920,NaN,NaN,S1F1_001,3057.0,S1F1_R1_02
5,2B,239.0,0.171429,NaN,NaN,S1F1_001,3043.0,S1F1_R1_02


#### Generating a function to add the unblinded compound data to the summary of the image analysis results

In [15]:
def add_Compound(row, metadata):
    wellID = row['WellNo'][1]

    if wellID == 'A':
        compound = metadata.loc[metadata['Plate ID']==row['Plate ID']]['Compound A']
    elif wellID == 'B':
        compound = metadata.loc[metadata['Plate ID']==row['Plate ID']]['Compound B']
    elif wellID == 'C':
        compound = metadata.loc[metadata['Plate ID']==row['Plate ID']]['Compound C']
    elif wellID == 'D':
        compound = metadata.loc[metadata['Plate ID']==row['Plate ID']]['Compound D']
    #print(compound)
    return compound.values[0]

ia_results['Compound'] = ia_results.apply(
    lambda row: add_Compound(row, all_md), axis=1)



C:\Users\Emily\miniconda3\envs\neuroplant\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Generating a function to add the strain data the summary of the image analysis results

In [16]:
def add_Strain(row, metadata):
    slotID = row['WellNo'][0]
    strain = metadata.loc[
        (metadata['Plate ID']==row['Plate ID']) & 
        (metadata['Scanner Slot:']==slotID)]['Strain Well A ']
    
    if len(strain) == 0:
        return 'Empty'
    else:
        return strain.values[0]

ia_results['Strain'] = ia_results.apply(
    lambda row: add_Strain(row, all_md), axis=1)


C:\Users\Emily\miniconda3\envs\neuroplant\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


Filtering out the replicates with fewer than 150 worms

In [17]:
filtered = ia_results.loc[ia_results['Total Worms'] >= 150]
print(len(filtered))

487


Due to the current liquid handler workflow, worms are added to wells whether they contain experimental conditions or not. Data for wells that contained no test compounds is removed below

In [18]:
filtered = filtered.loc[filtered['Compound'] != 'Empty']

In [19]:
filtered = filtered.sort_values(['Compound','Strain', 'File Name'], ascending=[True, True, True])


#### Defining functions to aggregate worm locations for each condition

In [20]:
S1F1_worm_locs_foldr = plb.Path('C:/Users/Emily/Documents/S1f1/124_ia/')
S1F2_worm_locs_foldr = plb.Path('C:/Users/Emily/Documents/S1F2/')

In [21]:
#conversion factors for dpi to mm
# 1 inch = 25.4mm
mm = 25.4
# 1200 pixels per 25.4mm
px_mm = 1200/mm

def all_xs(strn, df, loc):
# Need to create an empty dictionary to hold the values
    results_dict = {}
    compound = ''
    i=0
    
    df2 = df.loc[df['Strain'] == strn]

    for index, row in df2.iterrows():
        if row['Compound'] == compound:
            i += 1
            compound = row['Compound']
            if i < 3:
                pooled = get_worm_locs(row, loc, results_dict, strn)
            else:
                continue
        else:
            i = 0
            compound = row['Compound']
            pooled = get_worm_locs(row, loc, results_dict, strn)

    pooled_df = pd.DataFrame.from_dict(pooled)
    strain_df = pooled_df.apply(lambda x: -(x/px_mm)+32.5)

    strain_df = strain_df.drop(columns = ['Rosmarinic acid', '2-Nonylquinolin-4(1H)-one'])
    #strain_df.to_csv('C:/Users/Emily/Documents/S1F2/' + strn + '_xs.csv')
    return strain_df

In [22]:
def get_worm_locs(row, wrms, result_dict, strain): 

    fname = row['File Name']
    wellnum = row['WellNo']

    if row['Strain'] == strain:
        loc_fname =  wrms.joinpath('loc_' + fname + '_' + wellnum + '.csv')

        temp = pd.read_csv(loc_fname)
        compound = row['Compound']
        xs = temp['X']

        if compound in result_dict:
            result_dict[compound] = result_dict[compound].append(xs)
            result_dict[compound].reset_index(inplace=True, drop=True)
            #result_dict[compound] = result_dict[compound]+xs
        else:
            result_dict[compound]=xs

    return result_dict


#### Preparing the data so that we can input it into the delta-delta bootstrapping package

In [23]:
GNmm_df = all_xs('GN1077', filtered, S1F2_worm_locs_foldr)
PRmm_df = all_xs('PR678', filtered, S1F1_worm_locs_foldr)
CXmm_df = all_xs('CX10', filtered, S1F1_worm_locs_foldr)
sub_comps = list(GNmm_df.columns.values)
sub_comps.remove('DMSO')
comps = list(GNmm_df.columns.values)
dfs = [(GNmm_df, 'GN1077'), (PRmm_df,'PR678'), (CXmm_df, 'CX10')]

In [24]:
def mlt(stups):
    df = stups[0]
    strain = stups[1]
    melted = pd.melt(df, var_name='Compound', value_name='X').dropna()
    melted['Strain'] = strain
    return melted

In [25]:
n2 = pd.read_csv('/Users/Emily/Documents/S1/S1_xs3.csv', index_col=0)

ntomelt = n2.rename(columns={'HuperzineA':'(-)-Huperzine A'} )
ntomelt = ntomelt[ntomelt.columns.intersection(comps)]
n2_melt = pd.melt(ntomelt, var_name='Compound', value_name='X').dropna()
n2_melt['Strain'] = 'N2'

In [26]:
for d in dfs:
    m = mlt(d)
    n2_melt = n2_melt.append(m)


#### Calculating the bootstrapped delta-deltas for each genotype against N2

In [1]:
strn = ['GN1077', 'PR678', 'CX10']

res = pd.DataFrame()

for s in strn:
    i = 0
    ssub = n2_melt.loc[(n2_melt['Strain'] == 'N2') | (n2_melt['Strain'] == s)]
    for c in sub_comps:
        sub = ssub.loc[(ssub['Compound']=='DMSO') | (ssub['Compound'] == c)]
        
        
        unpaired_delta2 = db.load(data=sub, x=['Strain', 'Strain'], y='X', delta2=True, experiment='Compound', 
                                  experiment_label=['DMSO', c], x1_level = ['N2', s])
        dd = unpaired_delta2.mean_diff.delta_delta.to_dict()
        ddsub = {k: dd[k] for k in ('bca_high', 'bca_low', 'difference', 'test', 'control', )}
        ddf = pd.DataFrame(ddsub.items())
        idex = i
        ddf['idex'] = idex
        ddfp = ddf.pivot(columns=0, index='idex')
        ddfp['Strain'] = s
        i += 1
        res = res.append(ddfp)
#res.to_csv('/Users/Emily/Desktop/ReviewerResponses/mutVn2_dd.csv')

NameError: name 'pd' is not defined

#### Finding test conditions who's distributions are not distinguishable from N2

In [32]:
t = res.copy()
gn = t.loc[t['Strain'] == 'GN1077']
gn.columns = gn.columns.get_level_values(0)
same = gn.loc[(gn['bca_low'] < 0) & (gn['bca_high'] > 0)]
same

,bca_high,bca_low,control,difference,test,
idex,,,,,,
19,0.172083,-3.296425,DMSO,-1.585629,H2O,GN1077
26,2.427882,-0.994734,DMSO,0.734911,Methyl palmitate,GN1077
27,3.098137,-0.559129,DMSO,1.201707,Oleanolic Acid,GN1077
39,2.500986,-1.000902,DMSO,0.708445,Ursolic acid,GN1077


#### Extending the analysis to compare PR678 and CX10 against GN1077

In [34]:
against_GN = pd.DataFrame()
GNstrns = ['PR678', 'CX10']
for s in GNstrns:
    i = 0
    ssub = n2_melt.loc[(n2_melt['Strain'] == 'GN1077') | (n2_melt['Strain'] == s)]
    for c in sub_comps:
        sub = ssub.loc[(ssub['Compound']=='DMSO') | (ssub['Compound'] == c)]
        
        
        unpaired_delta2 = db.load(data=sub, x=['Strain', 'Strain'], y='X', delta2=True, experiment='Compound', 
                                  experiment_label=['DMSO', c], x1_level = [s, 'GN1077'])
        dd = unpaired_delta2.mean_diff.delta_delta.to_dict()
        
        ddsub = {k: dd[k] for k in ('bca_high', 'bca_low', 'difference', 'test', 'control')}
        ddf = pd.DataFrame(ddsub.items())
        idex = i
        ddf['idex'] = idex
        ddfp = ddf.pivot(columns=0, index='idex')
        ddfp['Strain'] = s
        i += 1
        against_GN = against_GN.append(ddfp)
#against_GN.to_csv('/Users/Emily/Desktop/ReviewerResponses/singleGN_dd.csv')

#### And finally looking at how the two single mutants compare to each other

In [37]:
singles = pd.DataFrame()


i = 0
ssub = n2_melt.loc[(n2_melt['Strain'] == 'PR678') | (n2_melt['Strain'] == 'CX10')]
for c in sub_comps:
    sub = ssub.loc[(ssub['Compound']=='DMSO') | (ssub['Compound'] == c)]


    unpaired_delta2 = db.load(data=sub, x=['Strain', 'Strain'], y='X', delta2=True, experiment='Compound', 
                              experiment_label=['DMSO', c], x1_level = ['CX10', 'PR678'])
    dd = unpaired_delta2.mean_diff.delta_delta.to_dict()

    ddsub = {k: dd[k] for k in ('bca_high', 'bca_low', 'difference', 'test', 'control')}
    ddf = pd.DataFrame(ddsub.items())
    idex = i
    ddf['idex'] = idex
    ddfp = ddf.pivot(columns=0, index='idex')
    ddfp['Strain'] = 'PR678'
    i += 1
    singles = singles.append(ddfp)

#singles.to_csv('/Users/Emily/Desktop/ReviewerResponses/ddSingles.csv')

In [29]:
cols = list(GNmm_df.columns)
cols.insert(0, cols.pop(cols.index('DMSO')))

temp = pd.DataFrame()
N2_all = pd.DataFrame()

strains = ['PR678', 'CX10', 'GN1077']
refs = ['DMSO']

for s in strains:
    if s == 'CX10':
        xs = CXmm_df
    elif s == 'PR678':
        xs = PRmm_df
    elif s == 'GN1077':
        xs = GNmm_df
    for c in cols:
        temp['N2'] = n2[c]
        temp[s] = xs[c]
        db_obj = db.load(temp, idx=(['N2', s]))
        hold = db_obj.mean_diff.results
        hold['Compound'] = c
        N2_all = N2_all.append(hold)
            
#N2_all.to_csv('C:/Users/Emily/Documents/S1F2/N2vMuts_mdiff_dmso.csv')
    

#### Calculating the bootstrapped mean differences for each strain (test vs DMSO)

In [38]:

def mean_diff_calc(df,strain , ref):
    cols = list(df.columns)
    cols.insert(0, cols.pop(cols.index(ref)))
    db_obj = db.load(df, idx=(cols))
    results_df = db_obj.mean_diff.results
    results_df['Strain'] = strain
    #results_df.to_csv('C:/Users/Emily/Documents/S1F2/'+ strain + '_' + ref 
    #                 + '_meandif.csv')
    return results_df

In [43]:
all_strains = pd.DataFrame()
refs = ['DMSO', 'H2O']

for s in strn:
    for r in refs:
        if s == 'CX10':
            xs = CXmm_df
        elif s == 'PR678':
            xs = PRmm_df
        elif s == 'GN1077':
            xs = GNmm_df
        df = mean_diff_calc(xs, s, r)
        all_strains = all_strains.append(df)

#all_strains.to_csv('C:/Users/Emily/Documents/S1F2/all_strains_nullrefs_mdiff_dmso.csv')

,control,test,control_N,test_N,effect_size,is_paired,difference,ci,bca_low,bca_high,...,pvalue_permutation,permutation_count,permutations_var,pvalue_welch,statistic_welch,pvalue_students_t,statistic_students_t,pvalue_mann_whitney,statistic_mann_whitney,Strain
0,DMSO,(-)-Huperzine A,915,596,mean difference,None,-0.434662,95,-1.590813,0.675229,...,0.4354,5000,"[0.3092578147497013, 0.3026705772677074, 0.306...",0.449688,0.756205,0.432042,0.785912,0.732891,275499.5,GN1077
1,DMSO,1-octanol,915,811,mean difference,None,-0.040677,95,-0.936400,0.852508,...,0.9356,5000,"[0.2078562273082063, 0.20811816058727547, 0.20...",0.928821,0.089342,0.929035,0.089071,0.392430,379871.0,GN1077
2,DMSO,"2,3-Dihydrobenzofuran",915,851,mean difference,None,-1.424593,95,-2.399113,-0.482385,...,0.0040,5000,"[0.2378994994808257, 0.2363103173251524, 0.237...",0.003587,2.916425,0.003461,2.927464,0.010270,416816.0,GN1077
3,DMSO,"2,5-Dihydroxybenzoic acid",915,1024,mean difference,None,-0.757694,95,-1.628896,0.081486,...,0.0852,5000,"[0.19616071198872043, 0.19707390039523226, 0.1...",0.086726,1.713805,0.086836,1.713198,0.124177,487403.0,GN1077
4,DMSO,2-Methyl-1-butanol,915,434,mean difference,None,-2.090040,95,-3.444072,-0.752076,...,0.0006,5000,"[0.3814131462895112, 0.3956215463435009, 0.378...",0.001973,3.106139,0.000746,3.379992,0.000070,225124.0,GN1077
